In [1]:
#import's
from dotenv import load_dotenv
import os
import csv
import requests
import time
from datetime import datetime, timedelta 
#Loading API key's from .env file
load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
IATA_CODE = "LAX" #LA
TYPE = "arrival"
#this code is so the API automatically pull's next's weeks flight schedule
start_date = datetime.today() + timedelta(days=8)
date_range = [start_date + timedelta(days=i) for i in range(7)]

In [2]:
all_flights_week = []
# this is date loop for the API so each day of the week get's called
for date in date_range:
    formatted_date = date.strftime("%Y-%m-%d")
    params = {
        "iataCode": IATA_CODE,
        "type": TYPE,
        "date": formatted_date,
        "access_key": api_key
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json().get("data", [])
        if data:
            for flight in data:
                flight["flight_date"] = formatted_date 

            all_flights_week.extend(data)
            print(f"✅ Retrieved {len(data)} flights")
        else:
            print(f"⚠️ No data returned for {formatted_date}")
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        
        break
# Time is a full minute between each loop because that has to with the rate limit, 
# the loop is also designed to break if it fails and then the data is supposed to be dropped
    
    time.sleep(60)







✅ Retrieved 2818 flights
✅ Retrieved 2619 flights
✅ Retrieved 2615 flights
✅ Retrieved 2815 flights
✅ Retrieved 2792 flights
✅ Retrieved 2595 flights
✅ Retrieved 2732 flights


In [3]:
import pandas as pd
from pandas import json_normalize

In [4]:
#it was a pretty clean JSON file so not diffcult to parse with a standard JSON_normalize

df = json_normalize(all_flights_week, sep='_')
columns_to_keep = [
    'flight_date',
    'weekday',
    'arrival_iataCode',
    'arrival_terminal',
    'arrival_scheduledTime',
    'departure_iataCode',
    'airline_name',
    'flight_iataNumber',
    'aircraft_modelText'
]

df_filtered = df[columns_to_keep]


df.head()

,weekday,flight_date,departure_iataCode,departure_icaoCode,departure_terminal,departure_gate,departure_scheduledTime,arrival_iataCode,arrival_icaoCode,arrival_terminal,...,airline_icaoCode,flight_number,flight_iataNumber,flight_icaoNumber,codeshared_airline_name,codeshared_airline_iataCode,codeshared_airline_icaoCode,codeshared_flight_number,codeshared_flight_iataNumber,codeshared_flight_icaoNumber
0,4,2025-08-14,sea,ksea,,,14:24,lax,klax,3,...,afr,8671,af8671,afr8671,delta air lines,dl,dal,2830,dl2830,dal2830
1,4,2025-08-14,sea,ksea,,,14:24,lax,klax,3,...,kal,3455,ke3455,kal3455,delta air lines,dl,dal,2830,dl2830,dal2830
2,4,2025-08-14,sea,ksea,,,14:24,lax,klax,3,...,sas,3644,sk3644,sas3644,delta air lines,dl,dal,2830,dl2830,dal2830
3,4,2025-08-14,sea,ksea,,,14:24,lax,klax,3,...,sva,6886,sv6886,sva6886,delta air lines,dl,dal,2830,dl2830,dal2830
4,4,2025-08-14,sea,ksea,,,14:24,lax,klax,3,...,vir,3213,vs3213,vir3213,delta air lines,dl,dal,2830,dl2830,dal2830


In [5]:
# then a time stamp is created to perserve a file with todays data, so it can be reference in historic sense
timestamp = datetime.now().strftime('%Y%m%d_%H%M')
raw_path = f"flight_data/raw/flight_arrivals_{timestamp}.csv"
df_filtered.to_csv(raw_path, index=False)

